<a href="https://colab.research.google.com/github/deniss97/Baron/blob/master/%D0%9E%D1%82%D0%B1%D0%BE%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV
from lightgbm import LGBMRanker
from sklearn.metrics import ndcg_score
import matplotlib.pyplot as plt

data = pd.read_csv('intern_task.csv')
data.describe()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
count,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,...,2.352580e+05,2.352580e+05,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000
mean,0.677869,14828.413401,1.911960,0.206233,1.189847,0.550272,1.960082,0.803964,0.097557,0.541174,...,4.763432e+02,1.046698e+04,0.070299,0.740710,11.067947,0.281747,0.027033,0.000269,22.457910,9.560379
std,0.830033,8193.945170,1.237374,0.579089,1.037233,0.790947,1.203534,0.339955,0.266035,0.419973,...,1.954840e+04,2.649584e+06,0.100721,0.352800,15.336861,0.392089,0.033351,0.002088,63.708018,14.347378
min,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8215.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.620522,0.000000,0.000000,0.006703,0.000000,4.000000,2.000000
50%,0.000000,14935.000000,2.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.500000,...,0.000000e+00,0.000000e+00,0.000000,0.926779,9.448900,0.000000,0.017761,0.000008,12.000000,5.500000
75%,1.000000,21580.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1.000000,...,1.000000e+00,4.000000e-01,0.142857,0.999613,16.146733,0.500000,0.034954,0.000109,28.000000,12.000000
max,4.000000,29995.000000,31.000000,18.000000,27.000000,9.000000,31.000000,1.000000,1.000000,1.000000,...,1.731553e+06,8.000000e+08,0.818182,1.000000,2506.799764,1.000000,0.711261,0.250000,15074.000000,1552.000000


In [123]:
print("Количество дубликатов:", data.duplicated().sum())
data = data.drop_duplicates()

print("Количество пропущенных значений по столбцам:\n", data.isnull().sum())
data.fillna(data.median(), inplace=True)

Количество дубликатов: 0
Количество пропущенных значений по столбцам:
 rank           0
query_id       0
feature_0      0
feature_1      0
feature_2      0
              ..
feature_139    0
feature_140    0
feature_141    0
feature_142    0
feature_143    0
Length: 146, dtype: int64


In [124]:
import optuna
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score
from lightgbm import LGBMRanker

def objective(trial):
    param = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True)
    }

    model = LGBMRanker(**param)
    model.fit(
        X_train, y_train,
        group=groups_train.groupby(groups_train).size().to_numpy(),
        eval_set=[(X_test, y_test)],
        eval_group=[groups_test.groupby(groups_test).size().to_numpy()],
        eval_metric='ndcg', eval_at=[5]
    )

    predictions = model.predict(X_test)
    ndcg = ndcg_score(np.asarray([y_test]), np.asarray([predictions]), k=5)
    return ndcg

# Presuming your data pre-processing code stays the same...
conditions = True
for i in range(144):
    feature_name = f'feature_{i}'
    q_low = data[feature_name].quantile(0.001)
    q_high = data[feature_name].quantile(0.999)
    conditions &= (data[feature_name] >= q_low) & (data[feature_name] <= q_high)


data = data[conditions]

print(len(data))
X = data.drop(['rank', 'query_id'], axis=1)
y = data['rank']

groups = data['query_id']

gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]

# Running the Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Output best parameters and re-train the model using those parameters
print('Best trial:', study.best_trial.params)
best_params = study.best_trial.params
best_model = LGBMRanker(**best_params)
best_model.fit(
    X_train, y_train,
    group=groups_train.groupby(groups_train).size().to_numpy(),
    eval_set=[(X_test, y_test)],
    eval_group=[groups_test.groupby(groups_test).size().to_numpy()],
    eval_metric='ndcg', eval_at=[5]
)

final_predictions = best_model.predict(X_test)
final_ndcg = ndcg_score(np.asarray([y_test]), np.asarray([final_predictions]), k=5)
print(f"Final NDCG@5 score: {final_ndcg:.4f}")


[I 2024-04-29 15:54:20,700] A new study created in memory with name: no-name-eda670ea-8f4f-4f4e-b139-de50db11d84f


226365
[LightGBM] [Warning] feature_fraction is set=0.6305612236438758, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6305612236438758
[LightGBM] [Warning] lambda_l2 is set=1.2567718150451241e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2567718150451241e-08
[LightGBM] [Warning] lambda_l1 is set=0.07122610696425168, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.07122610696425168
[LightGBM] [Warning] bagging_fraction is set=0.6295869738922888, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6295869738922888
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.6305612236438758, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6305612236438758
[LightGBM] [Warning] lambda_l2 is set=1.2567718150451241e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2567718150451241e-08
[LightGBM] 

[I 2024-04-29 15:54:46,809] Trial 0 finished with value: 0.6800273072886168 and parameters: {'learning_rate': 0.2960741423436491, 'num_leaves': 68, 'feature_fraction': 0.6305612236438758, 'bagging_fraction': 0.6295869738922888, 'bagging_freq': 1, 'min_child_samples': 92, 'lambda_l1': 0.07122610696425168, 'lambda_l2': 1.2567718150451241e-08}. Best is trial 0 with value: 0.6800273072886168.


[LightGBM] [Warning] feature_fraction is set=0.9447322165295355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9447322165295355
[LightGBM] [Warning] lambda_l2 is set=3.2356034222319527, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2356034222319527
[LightGBM] [Warning] lambda_l1 is set=1.7694528182771584e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.7694528182771584e-06
[LightGBM] [Warning] bagging_fraction is set=0.8278002616100191, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8278002616100191
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.9447322165295355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9447322165295355
[LightGBM] [Warning] lambda_l2 is set=3.2356034222319527, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2356034222319527
[LightGBM] [Warning] lambda_

[I 2024-04-29 15:55:37,924] Trial 1 finished with value: 0.6472260379105315 and parameters: {'learning_rate': 0.21666222665780258, 'num_leaves': 270, 'feature_fraction': 0.9447322165295355, 'bagging_fraction': 0.8278002616100191, 'bagging_freq': 3, 'min_child_samples': 19, 'lambda_l1': 1.7694528182771584e-06, 'lambda_l2': 3.2356034222319527}. Best is trial 0 with value: 0.6800273072886168.


...

[LightGBM] [Warning] feature_fraction is set=0.6197602930012877, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6197602930012877
[LightGBM] [Warning] lambda_l2 is set=1.483637553463959e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.483637553463959e-05
[LightGBM] [Warning] lambda_l1 is set=4.986181455965172e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.986181455965172e-05
[LightGBM] [Warning] bagging_fraction is set=0.9280808607493084, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9280808607493084
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.6197602930012877, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6197602930012877
[LightGBM] [Warning] lambda_l2 is set=1.483637553463959e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.483637553463959e-05
[LightGBM] [Warnin

[I 2024-04-29 17:01:37,293] Trial 98 finished with value: 0.7128285766667023 and parameters: {'learning_rate': 0.13100793529533897, 'num_leaves': 300, 'feature_fraction': 0.6197602930012877, 'bagging_fraction': 0.9280808607493084, 'bagging_freq': 3, 'min_child_samples': 49, 'lambda_l1': 4.986181455965172e-05, 'lambda_l2': 1.483637553463959e-05}. Best is trial 68 with value: 0.9343974612438289.


[LightGBM] [Warning] feature_fraction is set=0.8916237259999418, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8916237259999418
[LightGBM] [Warning] lambda_l2 is set=4.327230818532996e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.327230818532996e-05
[LightGBM] [Warning] lambda_l1 is set=0.001027540076427306, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001027540076427306
[LightGBM] [Warning] bagging_fraction is set=0.6793897007379985, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6793897007379985
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.8916237259999418, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8916237259999418
[LightGBM] [Warning] lambda_l2 is set=4.327230818532996e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.327230818532996e-05
[LightGBM] [Warning]

[I 2024-04-29 17:02:11,438] Trial 99 finished with value: 0.7925700737469461 and parameters: {'learning_rate': 0.046919301734962934, 'num_leaves': 110, 'feature_fraction': 0.8916237259999418, 'bagging_fraction': 0.6793897007379985, 'bagging_freq': 2, 'min_child_samples': 62, 'lambda_l1': 0.001027540076427306, 'lambda_l2': 4.327230818532996e-05}. Best is trial 68 with value: 0.9343974612438289.


Best trial: {'learning_rate': 0.19352984792919467, 'num_leaves': 276, 'feature_fraction': 0.8662533141261209, 'bagging_fraction': 0.8103932939337507, 'bagging_freq': 2, 'min_child_samples': 56, 'lambda_l1': 0.0051205847482619716, 'lambda_l2': 1.2245362144914043e-05}
[LightGBM] [Warning] feature_fraction is set=0.8662533141261209, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8662533141261209
[LightGBM] [Warning] lambda_l2 is set=1.2245362144914043e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2245362144914043e-05
[LightGBM] [Warning] lambda_l1 is set=0.0051205847482619716, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0051205847482619716
[LightGBM] [Warning] bagging_fraction is set=0.8103932939337507, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8103932939337507
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is se

In [126]:
print('Best trial:', study.best_trial.params)
# Best trial: {'learning_rate': 0.19352984792919467, 'num_leaves': 276,
#              'feature_fraction': 0.8662533141261209, 'bagging_fraction': 0.8103932939337507,
#              'bagging_freq': 2, 'min_child_samples': 56,
#              'lambda_l1': 0.0051205847482619716, 'lambda_l2': 1.2245362144914043e-05}
print(f"Final NDCG@5 score: {final_ndcg:.4f}")
# Final NDCG@5 score: 0.9344

Best trial: {'learning_rate': 0.19352984792919467, 'num_leaves': 276, 'feature_fraction': 0.8662533141261209, 'bagging_fraction': 0.8103932939337507, 'bagging_freq': 2, 'min_child_samples': 56, 'lambda_l1': 0.0051205847482619716, 'lambda_l2': 1.2245362144914043e-05}
Final NDCG@5 score: 0.9344
